<a href="https://colab.research.google.com/github/Storm00212/Introduction-to-machine-learning-models/blob/main/Violence_detection_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1. System & File Handling
import os
import zipfile
import random
import glob
import shutil
#2. Video Processing & Image Utilities
import cv2                # OpenCV for reading videos & extracting frames
import numpy as np
from PIL import Image     # Optional for additional image manipulation
#3. Data Analysis & Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Enable inline plotting for Colab
%matplotlib inline
#4. Machine Learning / Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#5. Deep Learning Layers and Tools
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, BatchNormalization
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
# 6. Model Evaluation Tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools
#  7. Progress & Logging
from tqdm import tqdm     # for progress bars
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Step 1: Dataset Loading and Directory Setup

# Import required libraries
import os
import shutil
import zipfile

# Clone the public GitHub repository that contains the violence detection dataset
!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git

# Change the current working directory to the cloned repository
os.chdir("A-Dataset-for-Automatic-Violence-Detection-in-Videos")

# Display the folder structure to confirm that the dataset is available
for root, dirs, files in os.walk(".", topdown=True):
    level = root.replace(os.getcwd(), '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

# Create organized folders for training and testing data
base_dir = "/content/violence_dataset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Define subdirectories for each class (violent and non-violent)
train_violent_dir = os.path.join(train_dir, "violent")
train_nonviolent_dir = os.path.join(train_dir, "non_violent")
test_violent_dir = os.path.join(test_dir, "violent")
test_nonviolent_dir = os.path.join(test_dir, "non_violent")

# Create all directories if they do not already exist
os.makedirs(train_violent_dir, exist_ok=True)
os.makedirs(train_nonviolent_dir, exist_ok=True)
os.makedirs(test_violent_dir, exist_ok=True)
os.makedirs(test_nonviolent_dir, exist_ok=True)

print("Directory structure ready.")


Cloning into 'A-Dataset-for-Automatic-Violence-Detection-in-Videos'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 376 (delta 3), reused 11 (delta 3), pack-reused 364 (from 1)
Receiving objects: 100% (376/376), 1.02 GiB | 57.45 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (355/355), done.
./
    readme.md
    .gitignore
    violence-detection-dataset/
        action-class-occurrences.csv
        violent-action-classes.csv
        nonviolent-action-classes.csv
        non-violent/
            cam2/
                37.mp4
                40.mp4
                36.mp4
                38.mp4
                25.mp4
                1.mp4
                27.mp4
                7.mp4
                46.mp4
                14.mp4
                5.mp4
                45.mp4
                9.mp4
                13.mp4
                48.mp4
                50.mp4
     